In [2]:
import pandas as pd 
import numpy as np 
import os

In [3]:
folder_path="Mediapipe_csv_output"

In [4]:
data_frane=[]

In [5]:
for file_name in os.listdir(folder_path):
    if file_name.endswith(".csv"):
        word_meaning=file_name.split('_')[0]
        file_path=os.path.join(folder_path,file_name)
        df=pd.read_csv(file_path)
        df['word']=word_meaning
        data_frane.append(df)
final=pd.concat(data_frane,ignore_index=True)
final


,pose_0_x,pose_0_y,pose_0_z,pose_0_visibility,pose_1_x,pose_1_y,pose_1_z,pose_1_visibility,pose_2_x,pose_2_y,...,right_hand_18_x,right_hand_18_y,right_hand_18_z,right_hand_19_x,right_hand_19_y,right_hand_19_z,right_hand_20_x,right_hand_20_y,right_hand_20_z,word
0,0.498307,0.597076,-0.606710,0.999984,0.508464,0.564415,-0.583276,0.999963,0.515995,0.563786,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,ABSCOND
1,0.498279,0.596022,-0.600280,0.999983,0.508525,0.563733,-0.577851,0.999961,0.516003,0.563310,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,ABSCOND
2,0.498312,0.595275,-0.580530,0.999983,0.508645,0.563150,-0.557377,0.999960,0.516038,0.562847,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,ABSCOND
3,0.498371,0.594620,-0.568087,0.999982,0.508723,0.562634,-0.544764,0.999958,0.516058,0.562404,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,ABSCOND
4,0.498440,0.594223,-0.571533,0.999981,0.508799,0.562267,-0.548652,0.999956,0.516081,0.562085,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,ABSCOND
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67426,0.537084,0.467278,-0.554316,0.996545,0.550541,0.443692,-0.519437,0.986121,0.557418,0.448450,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,WRONG
67427,0.536741,0.467621,-0.600482,0.996675,0.550430,0.443740,-0.568386,0.986407,0.557099,0.448429,...,0.393912,0.914119,-0.037799,0.401047,0.921260,-0.029267,0.403612,0.915940,-0.022153,WRONG
67428,0.536561,0.468375,-0.563149,0.996817,0.550381,0.443924,-0.535783,0.986792,0.556885,0.448447,...,0.410493,0.927779,-0.033568,0.415468,0.932268,-0.024187,0.415681,0.925296,-0.015931,WRONG
67429,0.536602,0.468447,-0.555374,0.996998,0.550378,0.443880,-0.523718,0.987465,0.556726,0.448260,...,0.438260,0.970443,-0.032302,0.442374,0.978156,-0.023837,0.440963,0.970424,-0.015725,WRONG


In [ ]:
final.to_csv("ISl_model.csv",index=False)

In [18]:
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import StratifiedKFold

# Load the dataset
df = pd.read_csv('ISl_model')  # Replace 'ISl_model' with your actual filename

# Separate features and target variable
X = df.drop('word', axis=1)
y = df['word']

# Encode the target variable (since it's a string type)
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded)

# Define the LightGBM classifier
model = lgb.LGBMClassifier(random_state=42)

# Define hyperparameter grid for tuning
param_grid = {
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'num_leaves': [20, 31, 50, 100],
    'max_depth': [-1, 5, 10, 20],
    'min_data_in_leaf': [10, 20, 50, 100],
    'boosting_type': ['gbdt', 'dart'],
    'n_estimators': [100, 200, 300],
}

# Set up cross-validation with stratification
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Perform RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=model,
    param_distributions=param_grid,
    n_iter=20,  # Adjust for more iterations
    scoring='accuracy',
    cv=cv,
    verbose=1,
    n_jobs=-1,
    random_state=42
)

# Fit the model to the training data
random_search.fit(X_train, y_train)

# Print the best hyperparameters
print("Best Hyperparameters:", random_search.best_params_)

# Predict using the best model
best_model = random_search.best_estimator_
y_pred = best_model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.4f}')
print('Classification Report:')
print(classification_report(y_test, y_pred, target_names=le.classes_))


Fitting 5 folds for each of 20 candidates, totalling 100 fits
Best Hyperparameters: {'num_leaves': 100, 'n_estimators': 200, 'min_data_in_leaf': 50, 'max_depth': 10, 'learning_rate': 0.05, 'boosting_type': 'dart'}
Accuracy: 0.8946
Classification Report:
                   precision    recall  f1-score   support

          ABSCOND       0.79      0.92      0.85        12
           AFFIRM       1.00      0.86      0.92        21
      AFGHANISTAN       1.00      0.91      0.95        22
           AFRICA       0.93      0.93      0.93        15
   ANDHRA PRADESH       1.00      0.69      0.82        13
        ARGENTINA       0.84      0.89      0.86        18
         BACKDOOR       0.89      0.67      0.76        12
           BACKUP       1.00      0.82      0.90        11
            BIHAR       0.94      0.88      0.91        17
        BLUETOOTH       0.89      1.00      0.94        16
           BUDGET       0.94      0.94      0.94        17
           CANADA       0.93      0.7

# If training LIGHTGBM model train with follwoing Hyper parameter 

## Best Hyperparameters: {'num_leaves': 100, 'n_estimators': 200, 'min_data_in_leaf': 50, 'max_depth': 10, 'learning_rate': 0.05, 'boosting_type': 'dart'}

In [2]:
import pandas as pd 

In [3]:
df = pd.read_csv('ISl_model.csv')  # Replace with your actual filename

# Remove labels with fewer than 3 occurrences
label_counts = df['word'].value_counts()
filtered_labels = label_counts[label_counts >= 3].index
df_filtered = df[df['word'].isin(filtered_labels)]

# Verify the filtered dataset
print(f"Original dataset size: {df.shape[0]}")
print(f"Filtered dataset size: {df_filtered.shape[0]}")

# Continue with data preprocessing...
X = df_filtered.drop('word', axis=1)
y = df_filtered['word']

Original dataset size: 67431
Filtered dataset size: 67431


In [ ]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report

# Load dataset
df = pd.read_csv("ISl_model.csv")

# Separate features and target
X = df.drop(columns=['word'])  # Replace 'target' with your actual column name
y = df['word']

# Encode string target labels to numerical values
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Initialize XGBoost classifier
xgb_model = xgb.XGBClassifier(objective='multi:softmax', num_class=len(label_encoder.classes_), random_state=42)

# Hyperparameter grid (adjusted for your dataset size)
param_grid = {
    'max_depth': [6, 8, 10],
    'learning_rate': [0.05, 0.1, 0.2],
    'n_estimators': [200, 300, 500],
    'subsample': [0.8, 0.9],
    'colsample_bytree': [0.6, 0.8],
    'gamma': [0, 1, 5],
    'min_child_weight': [1, 5, 10],
}

# Perform Grid Search with cross-validation
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid,
                           cv=3, scoring='accuracy', n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

# Best parameters and model evaluation
print("Best Hyperparameters:", grid_search.best_params_)

# Use the best estimator to predict
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

# Decode predictions back to original string labels
y_pred_decoded = label_encoder.inverse_transform(y_pred)
y_test_decoded = label_encoder.inverse_transform(y_test)

# Evaluate model performance
print("Accuracy on Test Set:", accuracy_score(y_test_decoded, y_pred_decoded))
print("Classification Report:\n", classification_report(y_test_decoded, y_pred_decoded))

